# Project Benson

## Exploring... and Aggregating

In [50]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels
import seaborn as sns
from numpy import linalg

import math
import patsy

from statsmodels.formula.api import ols

%matplotlib inline

In [51]:
!python -V

Python 3.6.3 :: Anaconda custom (64-bit)


In [52]:
print("Pandas version:",pd.__version__)
print("Numpy version:",np.__version__)

Pandas version: 0.20.3
Numpy version: 1.13.3


## Pick a week and play...

In [53]:
#week = '170422' # Week of 2017, April 15
#week = '170318' # Week of 2017, April 15
#datafile = 'turnstile_%s.txt' % week
#url = 'http://web.mta.info/developers/data/nyct/turnstile/%s' % datafile

# Specify location to store dataframes
mydir = '/home/joseph/ds/Projects/Project_Benson/Data'
#mydir =

#df_pickle = '%s/turnstile_%s.pkl' % (mydir,week)
#hourly_pickle = '%s/turnstile_%s_hourly.pkl' % (mydir,week)

In [54]:
name = '2015_3mo'
filename = '%s.csv' % name

# Correct that nonsense of the extra spaces after the last column header
# Switch to upper case to make the NY headers match the MTA headers
!head -1 /mnt/Data/T*2015.csv | sed -e 's/ *$//' | tr '[:lower:]' '[:upper:]' > $mydir/$filename
!grep ',0[3-5]/' /mnt/Data/T*2015.csv >> $mydir/$filename

In [55]:
url = '%s/%s' % (mydir,filename)
df_pickle = '%s/turnstile_%s.pkl' % (mydir,name)
hourly_pickle = '%s/turnstile_%s_hourly.pkl' % (mydir,name)

## Data input

I start with the data off the MTA site.  I want a DateTime column immediately.

In [56]:
df = pd.read_csv(url, parse_dates = [['DATE','TIME']])

# If we're just reading this in from the MTA site, we should do this correction now
# Comment this out otherwise
df.columns.values[-1] = 'EXITS'


Or, if desired, read in previous work...

In [57]:
#df = pd.read_pickle(df_pickle, compression='gzip')
#hourly = pd.read_pickle(hourly_pickle, compression='gzip')

## Data Processing

In [58]:
df.set_index(['UNIT','SCP','STATION','DATE_TIME'], inplace=True)

In [59]:
def resampler(x):
    """ A function to resample time series data - to be used in groupby apply."""
    return (x.set_index('DATE_TIME')       # Resample based on our timestamp
            .resample('1H')                # Set the desired time period here
            .mean()                        # The aggregate function used to create the new sampled rows.
            .interpolate()                 # With the new rows, use interpolate to create the data
            .diff()                        # Now, use diff to create deltas
           )

The raw data represents a pattern of polling the turnstiles on a 4-hour period.

So, setting the time period for resampling to 1 hour may permmit further data analysis on an hourly basis but with the introduction of errors since we don't truly have that granularity.  Setting the time period greater than 4 hours
drops data because the first in the series is set to N/A with the diff.

In [60]:
hourly = (df.reset_index(level=3)
 .groupby(level=[0,1,2])
 .apply(resampler)
)

## Data Cleaning

In [61]:
# The diff opeation created N/A entries.  Let's set those to zero.
hourly.fillna(0,inplace=True)

We know we need to clean or drop the "leftover" rows from the diff operation.

Things that remain:
* Some turnstiles are actually counting backwards, believe it or not.  So we'll use *abs* to correct that.
* When turnstiles roll over or are reset, these create enormous anomalous values after diff.  This is low frequency (way less than 1%), so let's just set those to zero.

In [62]:
# There remain anomalies from a variety of causes.  Let's clean things up...
cleanitup = lambda x: abs(x) if abs(x) < 1000 else np.nan
hourly["ENTRIES"] = hourly["ENTRIES"].map(cleanitup).interpolate()
hourly["EXITS"] = hourly["EXITS"].map(cleanitup).interpolate()

## Data Storage

Now, let's save our work...

In [63]:
df.to_pickle(df_pickle, compression='gzip')
hourly.to_pickle(hourly_pickle, compression='gzip')

Yay!